# Web Scraping SBS

## Herramientas

- Chromdriver
- Selenium
    - Selectors
    - Frames
    - Input
- Pandas
- For Loop
- While Loop

## ChromeDriver

In [1]:
# !pip install webdriver-manager
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options

## Selenium

In [2]:
#!pip install selenium
from selenium import webdriver

Iniciamos con dos primeras lineas que sirven para setear la forma de presentación de la pantalla. Luego, en una sola línea utilizamos ChromeDriver y Selenium

In [3]:
# Setting
options= Options()
options.add_argument( "--start-maximized" )
driver = webdriver.Chrome(ChromeDriverManager().install(),options=options)
# With this option we connect with the URL we desire
driver.get("https://www.sbs.gob.pe/app/pp/EstadisticasSAEEPortal/Paginas/TIActivaTipoCreditoEmpresa.aspx?tip=B#")



====== WebDriver manager ======
Current google-chrome version is 106.0.5249
Get LATEST chromedriver version for 106.0.5249 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/106.0.5249.61/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\Anzony\.wdm\drivers\chromedriver\win32\106.0.5249.61]


## Explorando Html

Empezaremos explorando la página y viendo cómo poder obtener la tabla default utilizando selenium. Para ello vamos a inspeccionar la página y ver que cada elemento tiene una forma de seleccionar. Toda página está hecha a través de HTML5 que permite maquetar la página y colocar cada cosa que vemos dentro de una caja. Caminando sobre los elementos podemos llegar a cada caja. Ejemplo:

In [6]:
driver.find_element_by_id( "ctl00_cphContent_btnExportar" ).get_attribute( "value" )

'Exportar'

In [7]:
driver.find_element_by_css_selector('#ctl00_cphContent_divConsultarDiario > table > tbody > tr > td > table > tbody > tr > td > table > tbody > tr > td.APLI_txtCampo > div').text

'Ingrese fecha: '

In [8]:
driver.find_element_by_xpath( "/html/body/form/div[4]/div/div/div[1]/table/tbody/tr/td/div/span").text

'TASA DE INTERÉS PROMEDIO DEL SISTEMA BANCARIO'

Podemos identificar los elementos a través de un ID, Selector, FullXpath o Xpath. Con ello identificamos cada elemento. Además, podemos interactuar con ellos. Dar click o inputear alguna información.

Click

In [9]:
driver.find_element_by_id("ctl00_cphContent_lbtnMex").click()
import time
time.sleep(3)

Input Value

In [12]:
from selenium.webdriver.common.keys import Keys

In [ ]:
driver.execute_script( f"document.getElementById('ctl00_cphContent_rdpDate_dateInput').value = '04/07/2007'")
driver.find_element_by_id("ctl00_cphContent_rdpDate_dateInput").send_keys(Keys.RETURN)
time.sleep(3)

driver.find_element_by_xpath("/html/body/form/table")

**¿Por qué no podemos obtener este elemento si existe en la maqueta de html?**


Porque este elemento esta dentro de otro html que a su vez está embebido a esta página web.

In [16]:
# Cambiamos de frame
driver.switch_to.frame(driver.find_element_by_id("ctl00_cphContent_pnContenidoAnteriorMnASP"))

In [17]:
# Seleccionamos la tabla
tabla = driver.find_element_by_xpath("/html/body/form/table")

In [20]:
driver.switch_to.default_content()
driver.find_element_by_id("ctl00_cphContent_btnConsultar")

<selenium.webdriver.remote.webelement.WebElement (session="13de3453e725908ffd5f7c62b25bf95a", element="ab71d3ba-4f6b-4231-9eb2-4cb8faa03973")>

### ¿Cómo lidiamos con las tablas?

La forma más fácil es usando pandas para leer html. Pandas te permite importar datos desde una estructura HTML. La estructura de las tablas cambia para fechas antiguas y para fechas modernas.

#### Antiguos

In [26]:
# Input date
driver.execute_script( f"document.getElementById('ctl00_cphContent_rdpDate_dateInput').value = '30/10/2015'")
driver.find_element_by_id("ctl00_cphContent_rdpDate_dateInput").send_keys(Keys.RETURN)
time.sleep(3)
# # Cambiamos de frame
driver.switch_to.frame(driver.find_element_by_id("ctl00_cphContent_pnContenidoAnteriorMnASP"))
driver.find_element_by_xpath("/html/body/form/table")
# Seleccionamos la tabla
tabla = driver.find_element_by_xpath("/html/body/form/table")

tabla_html = tabla.get_attribute('outerHTML')
# # Volvemos al frame principal
driver.switch_to.default_content()

In [34]:
import pandas as pd
tablas = pd.read_html(tabla_html)

In [ ]:
tabla_reducida = tablas[2].iloc[2:, :]
tabla_reducida

In [38]:
tabla_reducida.to_excel( r"C:\Users\Anzony\Downloads/file.xlsx", index = False, header= False )

### Recientes 

In [41]:
# Input date
driver.execute_script( f"document.getElementById('ctl00_cphContent_rdpDate_dateInput').value = '30/11/2015'")
driver.find_element_by_id("ctl00_cphContent_rdpDate_dateInput").send_keys(Keys.RETURN)
time.sleep(4)

# Identificar directamente la tabla
# Seleccionamos la tabla
tabla = driver.find_element_by_id("ctl00_cphContent_rpgActualMn_OT")
tabla_html = tabla.get_attribute('outerHTML')


In [236]:
import numpy as np

In [50]:
table0.iloc[30:, :]

,Continental,Comercio,Crédito,Financiero,BIF,Scotiabank,Citibank,Interbank,Mibanco,GNB,Falabella,Santander,Ripley,Azteca,Deutsche,CAT,ICBC,Promedio,Unnamed: 18
30,24.49,ContinentalComercioCréditoFinancieroBIFScotiab...,36.15,27.60,33.78,15.09,24.68,-,22.94,37.27,-,4.50,-,-,-,-,-,-,34.37
31,36.17,ContinentalComercioCréditoFinancieroBIFScotiab...,29.74,29.99,64.84,-,34.02,-,-,-,-,-,-,-,-,-,-,-,30.76
32,19.47,ContinentalComercioCréditoFinancieroBIFScotiab...,-,11.32,15.50,20.00,21.45,-,9.89,-,-,-,-,-,-,-,-,-,18.14
33,-,ContinentalComercioCréditoFinancieroBIFScotiab...,-,-,-,13.49,-,-,20.81,-,-,-,-,-,-,-,-,-,19.16
34,20.06,ContinentalComercioCréditoFinancieroBIFScotiab...,-,-,39.63,-,26.22,-,-,40.60,-,4.50,-,-,-,-,-,-,17.46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,Préstamos no Revolventes para libre disponibil...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90,Préstamos no Revolventes para libre disponibil...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91,Créditos pignoraticios,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92,Hipotecarios,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
import numpy as np

In [54]:
# Number of rows of second table
tb1_rows = table1.shape[0]
drop_rows = np.arange( 1, tb1_rows + 2 )
drop_rows

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47])

In [ ]:
# Clean first table
table0 = table0.drop( drop_rows, axis = 0 )
table0

In [ ]:
# Assign values to new table
col_name = table0.iloc[0, 0]
col_values = table0.iloc[1:, 0].tolist()
table_clean.insert( loc = 0, column= col_name, value = col_values)
table_clean1 = table_clean.copy()
table_clean1

Conseguimos exportar nuestra primera tabla. Ahora, tenemos que hacer esto para todas las últimsa fechas de cada mes donde haya disponible información de la SBS. Por lo tanto, tenemos que introducir información en el input de fecha y hacer la consulta para cada fecha.

**¿Cómo conseguimos las última fechas de cada mes?**

Vamos a utilizar la herramienta datetime de pandas.

In [65]:
# Generamos un rango de fechas
data_date = pd.DataFrame( {'date_real' : pd.date_range(start="01-01-2004",end="08-31-2022") })
data_date

,date_real
0,2004-01-01
1,2004-01-02
2,2004-01-03
3,2004-01-04
4,2004-01-05
...,...
6813,2022-08-27
6814,2022-08-28
6815,2022-08-29
6816,2022-08-30


In [66]:
# Les damos un formato y separamos los dias meses y años
data_date['date_info'] = data_date.date_real.dt.strftime("%d/%m/%Y")
data_date
data_split = data_date.date_info.str.split("/", expand= True ).astype(int)
data_split.columns = ["Day", "Month", "Year"]
data_date = pd.concat([ data_date, data_split ], axis = 1 )
data_date

,date_real,date_info,Day,Month,Year
0,2004-01-01,01/01/2004,1,1,2004
1,2004-01-02,02/01/2004,2,1,2004
2,2004-01-03,03/01/2004,3,1,2004
3,2004-01-04,04/01/2004,4,1,2004
4,2004-01-05,05/01/2004,5,1,2004
...,...,...,...,...,...
6813,2022-08-27,27/08/2022,27,8,2022
6814,2022-08-28,28/08/2022,28,8,2022
6815,2022-08-29,29/08/2022,29,8,2022
6816,2022-08-30,30/08/2022,30,8,2022


In [67]:
# Tomamos la última fecha
data_date['date_real'] = pd.to_datetime(data_date.date_info, format="%d/%m/%Y")
# Get the last 5 days per month
final_date_data = data_date.sort_values('date_real').groupby([ "Year" , "Month" ]).tail(1).reset_index(drop = True)
dates_list = final_date_data.date_real.tolist()
dates_list

[Timestamp('2004-01-31 00:00:00'),
 Timestamp('2004-02-29 00:00:00'),
 Timestamp('2004-03-31 00:00:00'),
 Timestamp('2004-04-30 00:00:00'),
 Timestamp('2004-05-31 00:00:00'),
 Timestamp('2004-06-30 00:00:00'),
 Timestamp('2004-07-31 00:00:00'),
 Timestamp('2004-08-31 00:00:00'),
 Timestamp('2004-09-30 00:00:00'),
 Timestamp('2004-10-31 00:00:00'),
 Timestamp('2004-11-30 00:00:00'),
 Timestamp('2004-12-31 00:00:00'),
 Timestamp('2005-01-31 00:00:00'),
 Timestamp('2005-02-28 00:00:00'),
 Timestamp('2005-03-31 00:00:00'),
 Timestamp('2005-04-30 00:00:00'),
 Timestamp('2005-05-31 00:00:00'),
 Timestamp('2005-06-30 00:00:00'),
 Timestamp('2005-07-31 00:00:00'),
 Timestamp('2005-08-31 00:00:00'),
 Timestamp('2005-09-30 00:00:00'),
 Timestamp('2005-10-31 00:00:00'),
 Timestamp('2005-11-30 00:00:00'),
 Timestamp('2005-12-31 00:00:00'),
 Timestamp('2006-01-31 00:00:00'),
 Timestamp('2006-02-28 00:00:00'),
 Timestamp('2006-03-31 00:00:00'),
 Timestamp('2006-04-30 00:00:00'),
 Timestamp('2006-05-

Tenemos una lista de los último días de cada mes desde 2000 al 2022. La idea es empezar en este día e ir probando hacia atrás hasta encontrarnos con el último día en el que se haya reportado información. Para hacer este proceso iterativo utilizaremos un while loop.

While loop es un proceso iterativo que se repite hasta que cierta condición se cumple. Para nuestro caso, hasta que hayamos encontrado una tabla. Ahora, debemos tener en cuenta que el driver puede enfrentarse a dos tipos de tablas como las que ya vimos arriba. Todo esto lo tendremos que tomar en cuenta.

Primero, iniciemos con una fecha última la cual no tiene tabla "31/07/2022".

In [68]:
import time

# 
# le damos el fomato que queremos
last_day = dates_list[-2]
last_day_str = last_day.strftime("%d/%m/%Y")
print(last_day_str)

# repetimos el proceso de obtener tablas
driver.execute_script( f"document.getElementById('ctl00_cphContent_rdpDate_dateInput').value = '{last_day_str}'")
driver.find_element_by_id("ctl00_cphContent_rdpDate_dateInput").send_keys(Keys.RETURN)
time.sleep( 2 )

31/07/2022


Lo que hariamos es ir a un dia anterior "30/07/2022".

In [69]:
# Vamos un dia anterior
from datetime import datetime, timedelta
last_day = last_day - timedelta(days= 1)
last_day_str = last_day.strftime("%d/%m/%Y")
print(last_day_str)


# repetimos el proceso de obtener tablas
driver.execute_script( f"document.getElementById('ctl00_cphContent_rdpDate_dateInput').value = '{last_day}'")
driver.find_element_by_id("ctl00_cphContent_rdpDate_dateInput").send_keys(Keys.RETURN)
time.sleep( 2 )

30/07/2022


Como esta fecha tampoco tiene tabla tenemos que ir a un día anterior "29/07/2022".

In [70]:
# Vamos un dia anterior
from datetime import datetime, timedelta
last_day = last_day - timedelta(days= 1)
last_day_str = last_day.strftime("%d/%m/%Y")
print(last_day_str)


# repetimos el proceso de obtener tablas
driver.execute_script( f"document.getElementById('ctl00_cphContent_rdpDate_dateInput').value = '{last_day}'")
driver.find_element_by_id("ctl00_cphContent_rdpDate_dateInput").send_keys(Keys.RETURN)
time.sleep( 2 )

29/07/2022


Una vez más "28/07/2022".

In [71]:
# Vamos un dia anterior
from datetime import datetime, timedelta
last_day = last_day - timedelta(days= 1)
last_day_str = last_day.strftime("%d/%m/%Y")
print(last_day_str)


# repetimos el proceso de obtener tablas
driver.execute_script( f"document.getElementById('ctl00_cphContent_rdpDate_dateInput').value = '{last_day}'")
driver.find_element_by_id("ctl00_cphContent_rdpDate_dateInput").send_keys(Keys.RETURN)
time.sleep( 2 )

28/07/2022


Una vez más "27/07/2022".

In [72]:
# Vamos un dia anterior
from datetime import datetime, timedelta
last_day = last_day - timedelta(days= 1)
last_day_str = last_day.strftime("%d/%m/%Y")
print(last_day_str)


# repetimos el proceso de obtener tablas
driver.execute_script( f"document.getElementById('ctl00_cphContent_rdpDate_dateInput').value = '{last_day}'")
driver.find_element_by_id("ctl00_cphContent_rdpDate_dateInput").send_keys(Keys.RETURN)
time.sleep( 2 )

27/07/2022


Aquí, sí encontramos información. Entonces, el scrpaer tiene que recorrer hacia atrás hasta encontrar una tabla. Esto lo podemos hacer utilizando los ojos, pero como lo haría el scraper. Bueno, provando si existe una table como las antiguas o las recientes. Para eso utilizaremos `try`.

In [75]:
# le damos el fomato que queremos
last_day_str = "31/07/2022"
print(last_day_str)

# repetimos el proceso de obtener tablas
driver.execute_script( f"document.getElementById('ctl00_cphContent_rdpDate_dateInput').value = '{last_day_str}'")
driver.find_element_by_id("ctl00_cphContent_rdpDate_dateInput").send_keys(Keys.RETURN)
time.sleep( 2 )

# busquems la tabla
# prueba si es una tabla tipo antigua
try:
    # Cambiamos de frame
    driver.switch_to.frame(driver.find_element_by_id("ctl00_cphContent_pnContenidoAnteriorMnASP"))
    driver.find_element_by_xpath("/html/body/form/table")
    # Seleccionamos la tabla
    tabla = driver.find_element_by_xpath("/html/body/form/table")

    tabla_html = tabla.get_attribute('outerHTML')
    # Volvemos al frame principal
    driver.switch_to.default_content()
    tablas = pd.read_html(tabla_html)
    tabla_reducida = tablas[2].iloc[2:, :]
    tabla_reducida
    # Dame el numero de columnas
    print( "Tenemos una tabla tipo antigua")
except:
    print( "No Tenemos una tabla tipo antigua")

31/07/2022
No Tenemos una tabla tipo antigua


In [77]:
# Probemos una tabla reciente
# le damos el fomato que queremos
last_day_str = "31/07/2022"
print(last_day_str)

# repetimos el proceso de obtener tablas
driver.execute_script( f"document.getElementById('ctl00_cphContent_rdpDate_dateInput').value = '{last_day_str}'")
driver.find_element_by_id("ctl00_cphContent_rdpDate_dateInput").send_keys(Keys.RETURN)
time.sleep( 5 )
try:

    # Identificar directamente la tabla
    # Seleccionamos la tabla
    tabla = driver.find_element_by_id("ctl00_cphContent_rpgActualMn_OT")
    tabla_html = tabla.get_attribute('outerHTML')
    tablas = pd.read_html(tabla_html)
    tablas
    # generate table objects
    table0, table1 = tablas[0], tablas[1]
    table_clean = table1.copy()

    # Number of rows of second table
    tb1_rows = table1.shape[0]
    drop_rows = np.arange( 1, tb1_rows + 2 )

    # Clean first table
    table0 = table0.drop( drop_rows, axis = 0 )

    # Assign values to new table
    col_name = table0.iloc[0, 0]
    col_values = table0.iloc[1:, 0].tolist()
    table_clean.insert( loc = 0, column= col_name, value = col_values)
    table_clean1 = table_clean.copy()
    table_clean1
    print("Tenemos Una tabla Reciente")
except:
    print("No Tenemos Una tabla Reciente")

31/07/2022
No Tenemos Una tabla Reciente


Como no tenemos ni tabla reciente, ni antigua, entonces probemos con el dia que si sabemos que hay tabla

In [78]:
# le damos el fomato que queremos
last_day_str = "27/07/2022"
print(last_day_str)

# repetimos el proceso de obtener tablas
driver.execute_script( f"document.getElementById('ctl00_cphContent_rdpDate_dateInput').value = '{last_day_str}'")
driver.find_element_by_id("ctl00_cphContent_rdpDate_dateInput").send_keys(Keys.RETURN)
time.sleep( 3 )

# busquems la tabla
# prueba si es una tabla tipo antigua
try:
    # Cambiamos de frame
    driver.switch_to.frame(driver.find_element_by_id("ctl00_cphContent_pnContenidoAnteriorMnASP"))
    driver.find_element_by_xpath("/html/body/form/table")
    # Seleccionamos la tabla
    tabla = driver.find_element_by_xpath("/html/body/form/table")

    tabla_html = tabla.get_attribute('outerHTML')
    # Volvemos al frame principal
    driver.switch_to.default_content()
    tablas = pd.read_html(tabla_html)
    tabla_reducida = tablas[2].iloc[2:, :]
    tabla_reducida
    # Dame el numero de columnas
    print( "Tenemos una tabla tipo antigua")
except:
    print( "No Tenemos una tabla tipo antigua")
    

try:

    # Identificar directamente la tabla
    # Seleccionamos la tabla
    tabla = driver.find_element_by_id("ctl00_cphContent_rpgActualMn_OT")
    tabla_html = tabla.get_attribute('outerHTML')
    tablas = pd.read_html(tabla_html)
    tablas
    # generate table objects
    table0, table1 = tablas[0], tablas[1]
    table_clean = table1.copy()

    # Number of rows of second table
    tb1_rows = table1.shape[0]
    drop_rows = np.arange( 1, tb1_rows + 2 )

    # Clean first table
    table0 = table0.drop( drop_rows, axis = 0 )

    # Assign values to new table
    col_name = table0.iloc[0, 0]
    col_values = table0.iloc[1:, 0].tolist()
    table_clean.insert( loc = 0, column= col_name, value = col_values)
    table_clean1 = table_clean.copy()
    table_clean1
    print("Tenemos Una tabla Reciente")
except:
    print("No Tenemos Una tabla Reciente")

27/07/2022
No Tenemos una tabla tipo antigua
Tenemos Una tabla Reciente


Bien, ya vimos que pudimos identificar las tablas. Ahora, utilicemos el while loop para hacer esto automaticamente. Vamos a permitirle al driver que retroceda hasta una semana.

In [79]:
days_before = 0
last_day = dates_list[-2]
while days_before < 8:
    
    # le damos el fomato que queremos
    last_day_str = last_day.strftime("%d/%m/%Y")
    print(last_day_str)

    # repetimos el proceso de obtener tablas
    driver.execute_script( f"document.getElementById('ctl00_cphContent_rdpDate_dateInput').value = '{last_day_str}'")
    driver.find_element_by_id("ctl00_cphContent_rdpDate_dateInput").send_keys(Keys.RETURN)
    time.sleep( 4 )

    # busquems la tabla
    # prueba si es una tabla tipo antigua
    try:
        # Cambiamos de frame
        driver.switch_to.frame(driver.find_element_by_id("ctl00_cphContent_pnContenidoAnteriorMnASP"))
        driver.find_element_by_xpath("/html/body/form/table")
        # Seleccionamos la tabla
        tabla = driver.find_element_by_xpath("/html/body/form/table")

        tabla_html = tabla.get_attribute('outerHTML')
        # Volvemos al frame principal
        driver.switch_to.default_content()
        tablas = pd.read_html(tabla_html)
        tabla_antigua = tablas[2].iloc[2:, :]
        tabla_antigua
        # Dame el numero de columnas
        print( "Tenemos una tabla tipo antigua")
    except:
        print( "No Tenemos una tabla tipo antigua")
        tabla_antigua = ""


    try:

        # Identificar directamente la tabla
        # Seleccionamos la tabla
        tabla = driver.find_element_by_id("ctl00_cphContent_rpgActualMn_OT")
        tabla_html = tabla.get_attribute('outerHTML')
        tablas = pd.read_html(tabla_html)
        tablas
        # generate table objects
        table0, table1 = tablas[0], tablas[1]
        table_clean = table1.copy()

        # Number of rows of second table
        tb1_rows = table1.shape[0]
        drop_rows = np.arange( 1, tb1_rows + 2 )

        # Clean first table
        table0 = table0.drop( drop_rows, axis = 0 )

        # Assign values to new table
        col_name = table0.iloc[0, 0]
        col_values = table0.iloc[1:, 0].tolist()
        table_clean.insert( loc = 0, column= col_name, value = col_values)
        tabla_reciente = table_clean.copy()
        tabla_reciente
        print("Tenemos Una tabla Reciente")
    except:
        print("No Tenemos Una tabla Reciente")
        tabla_reciente = ""
        

    if ( ( isinstance( tabla_reciente, str ) == True ) and ( isinstance( tabla_antigua, str ) == True ) ):
        days_before = 1 + days_before
        last_day = last_day - timedelta(days= 1)
        
    elif ( isinstance( tabla_reciente, str ) == False ) and ( tabla_reciente.shape[1] > 5 ) :
        days_before = 8 
    
    elif ( isinstance( tabla_antigua, str ) == False ) and ( tabla_antigua.shape[1] > 5 ):
        days_before = 8 

31/07/2022
No Tenemos una tabla tipo antigua
No Tenemos Una tabla Reciente
30/07/2022
No Tenemos una tabla tipo antigua
No Tenemos Una tabla Reciente
29/07/2022
No Tenemos una tabla tipo antigua
No Tenemos Una tabla Reciente
28/07/2022
No Tenemos una tabla tipo antigua
No Tenemos Una tabla Reciente
27/07/2022
No Tenemos una tabla tipo antigua
Tenemos Una tabla Reciente


Encontramos la tabla y discrimanamos por tipo de tabla. Lo que faltaría es replicar este procedimiento para todas las últimas fechas que tenemos en la lista y esto lo hacemos utilizando el for loop.

In [80]:
for last_day in dates_list:
    days_before = 0
    
    while days_before < 8:
        driver.switch_to.default_content()
        # le damos el fomato que queremos
        last_day_str = last_day.strftime("%d/%m/%Y")
        print(last_day_str)

        # repetimos el proceso de obtener tablas
        driver.execute_script( f"document.getElementById('ctl00_cphContent_rdpDate_dateInput').value = '{last_day_str}'")
        driver.find_element_by_id("ctl00_cphContent_rdpDate_dateInput").send_keys(Keys.RETURN)
        time.sleep( 5 )

        # busquems la tabla
        # prueba si es una tabla tipo antigua
        try:
            # Cambiamos de frame
            driver.switch_to.frame(driver.find_element_by_id("ctl00_cphContent_pnContenidoAnteriorMnASP"))
            driver.find_element_by_xpath("/html/body/form/table")
            # Seleccionamos la tabla
            tabla = driver.find_element_by_xpath("/html/body/form/table")

            tabla_html = tabla.get_attribute('outerHTML')
            # Volvemos al frame principal
            driver.switch_to.default_content()
            tablas = pd.read_html(tabla_html)
            tabla_antigua = tablas[2].iloc[2:, :]
            tabla_antigua
            # Dame el numero de columnas
            print( "Tenemos una tabla tipo antigua")
        except:
            print( "No Tenemos una tabla tipo antigua")
            tabla_antigua = ""


        try:

            # Identificar directamente la tabla
            # Seleccionamos la tabla
            tabla = driver.find_element_by_id("ctl00_cphContent_rpgActualMn_OT")
            tabla_html = tabla.get_attribute('outerHTML')
            tablas = pd.read_html(tabla_html)
            tablas
            # generate table objects
            table0, table1 = tablas[0], tablas[1]
            table_clean = table1.copy()

            # Number of rows of second table
            tb1_rows = table1.shape[0]
            drop_rows = np.arange( 1, tb1_rows + 2 )

            # Clean first table
            table0 = table0.drop( drop_rows, axis = 0 )

            # Assign values to new table
            col_name = table0.iloc[0, 0]
            col_values = table0.iloc[1:, 0].tolist()
            table_clean.insert( loc = 0, column= col_name, value = col_values)
            tabla_reciente = table_clean.copy()
            tabla_reciente
            print("Tenemos Una tabla Reciente")
        except:
            print("No Tenemos Una tabla Reciente")
            tabla_reciente = ""


        if ( ( isinstance( tabla_reciente, str ) == False ) and ( tabla_reciente.shape[1] > 5 ) ):
            days_before = 8 
            print( tabla_reciente.columns.tolist() )

        elif ( ( isinstance( tabla_antigua, str ) == False ) and ( tabla_antigua.shape[1] > 5 ) ):
            days_before = 8 
            print( tabla_antigua.iloc[ :2, : ] )

        else:
            print( "la tabla no cumple con tener más de 5 columnas.")
            days_before = days_before + 1
            last_day = last_day - timedelta(days= 1)
            print( f"GO {days_before} before")

31/01/2004
Tenemos una tabla tipo antigua
No Tenemos Una tabla Reciente
la tabla no cumple con tener más de 5 columnas.
GO 1 before
30/01/2004
Tenemos una tabla tipo antigua
No Tenemos Una tabla Reciente
    0                         1           2   \
2  NaN                 COMERCIAL   COMERCIAL   
3  NaN  Avances en Cta.Corriente  Sobregiros   

                                  3                                 4   \
2                          COMERCIAL                         COMERCIAL   
3  Dsctos. y préstamos hasta 30 días  Dsctos. y préstamos 31 - 90 días   

                                  5                                   6   \
2                          COMERCIAL                           COMERCIAL   
3  Dsctos. y préstamos 91 - 180 días  Dsctos. y préstamos 181 - 360 días   

                                 7                 8   \
2                         COMERCIAL  A MICROEMPRESAS2   
3  Dsctos. y préstamos más 360 días        Sobregiros   

                           

KeyboardInterrupt: 